In [ ]:
import pandas as pd
import sqlalchemy
import numpy as np

In [ ]:
db_path = 'indian_food.sqlite'
csv_path = 'indian_food.csv'

In [ ]:
engine = sqlalchemy.create_engine('sqlite+pysqlite:///' + db_path)
metadata = sqlalchemy.MetaData()

In [ ]:
df = pd.read_csv(csv_path)
df.replace(-1, np.nan, inplace=True)
df.replace('-1', np.nan, inplace=True)
df.dropna(inplace=True)

In [ ]:
dump = ['''
-- Table: Course
CREATE TABLE IF NOT EXISTS Course (
    course_id integer NOT NULL CONSTRAINT Course_pk PRIMARY KEY,
    course_name varchar(12) NOT NULL
);
''',
'''
-- Table: Diet
CREATE TABLE IF NOT EXISTS Diet (
    diet_id integer NOT NULL CONSTRAINT Diet_pk PRIMARY KEY,
    diet_name character(15) NOT NULL
);
''',
'''
-- Table: Flavor_profile
CREATE TABLE IF NOT EXISTS Flavor_profile (
    flavor_id integer NOT NULL CONSTRAINT Flavor_profile_pk PRIMARY KEY,
    flavor_name varchar(7) NOT NULL
);
''',
'''
-- Table: Food
CREATE TABLE IF NOT EXISTS Food (
    food_id integer NOT NULL CONSTRAINT Food_pk PRIMARY KEY,
    food_name varchar(30) NOT NULL,
    ingredients varchar(100) NOT NULL,
    prep_time integer NOT NULL,
    cook_time integer NOT NULL,
    state_id integer NOT NULL,
    region_id integer NOT NULL,
    diet_id integer NOT NULL,
    flavor_id integer NOT NULL,
    course_id integer NOT NULL,
    CONSTRAINT Food_State FOREIGN KEY (state_id)
    REFERENCES State (state_id),
    CONSTRAINT Food_Region FOREIGN KEY (region_id)
    REFERENCES Region (region_id),
    CONSTRAINT Food_Diet FOREIGN KEY (diet_id)
    REFERENCES Diet (diet_id),
    CONSTRAINT Food_Flavor_profile FOREIGN KEY (flavor_id)
    REFERENCES Flavor_profile (flavor_id),
    CONSTRAINT Food_Course FOREIGN KEY (course_id)
    REFERENCES Course (course_id)
);
''',
'''
-- Table: Region
CREATE TABLE IF NOT EXISTS Region (
    region_id integer NOT NULL CONSTRAINT Region_pk PRIMARY KEY,
    region_name varchar(11) NOT NULL
);
''',
'''
-- Table: State
CREATE TABLE IF NOT EXISTS State (
    state_id integer NOT NULL CONSTRAINT State_pk PRIMARY KEY,
    state_name varchar(15) NOT NULL,
    region_id integer NOT NULL,
    CONSTRAINT State_Region FOREIGN KEY (region_id)
    REFERENCES Region (region_id)
);
'''
]


In [ ]:
with engine.connect() as conn:
    for statement in dump:
        conn.execute(statement)

In [ ]:
values = df['diet'].unique()
tablename = 'Diet'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values:
        conn.execute(query, {'diet_name': v})

In [ ]:
with engine.connect() as conn:
    query = sqlalchemy.select([metadata.tables['Diet']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'vegetarian')
(2, 'non vegetarian')


In [ ]:
values = df['flavor_profile'].unique()
tablename = 'Flavor_profile'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values:
        conn.execute(query, {'flavor_name': v})

In [ ]:
with engine.connect() as conn:
    query = sqlalchemy.select([metadata.tables['Flavor_profile']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'sweet')
(2, 'spicy')
(3, 'bitter')
(4, 'sour')


In [ ]:
values = df['course'].unique()
tablename = 'Course'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values:
        conn.execute(query, {'course_name': v})

In [ ]:
with engine.connect() as conn:
    query = sqlalchemy.select([metadata.tables['Course']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'dessert')
(2, 'main course')
(3, 'starter')
(4, 'snack')


In [ ]:
values = df['region'].unique()
tablename = 'Region'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values:
        conn.execute(query, {'region_name': v})

In [ ]:
with engine.connect() as conn:
    query = sqlalchemy.select([metadata.tables['Region']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'East')
(2, 'West')
(3, 'North')
(4, 'North East')
(5, 'South')
(6, 'Central')


In [ ]:
def getId(table, item, col_id=None, col_name=None):
    if col_id is None:
        col_id = table.lower() + '_id'
    if col_name is None:
        col_name = table.lower() + '_name'
    table = metadata.tables[table]
    query = sqlalchemy.select([table.c[col_id]]).select_from(table).where(table.c[col_name] == item)
    with engine.connect() as conn:
        result = conn.execute(query).fetchone()
    return result[0]

In [ ]:
values = df[['state', 'region']].drop_duplicates()
tablename = 'State'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values.values:
        state_name = v[0]
        region_id = int(getId('Region', v[1]))
        conn.execute(query, {'state_name': state_name, 'region_id': region_id})

In [ ]:
with engine.connect() as conn: 
    query = sqlalchemy.select([metadata.tables['State']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'West Bengal', 1)
(2, 'Rajasthan', 2)
(3, 'Punjab', 3)
(4, 'Uttar Pradesh', 3)
(5, 'Odisha', 1)
(6, 'Maharashtra', 2)
(7, 'Uttarakhand', 3)
(8, 'Assam', 4)
(9, 'Bihar', 3)
(10, 'Andhra Pradesh', 5)
(11, 'Karnataka', 5)
(12, 'Telangana', 5)
(13, 'Kerala', 5)
(14, 'Tamil Nadu', 5)
(15, 'Gujarat', 2)
(16, 'Manipur', 4)
(17, 'Nagaland', 4)
(18, 'NCT of Delhi', 3)
(19, 'Jammu & Kashmir', 3)
(20, 'Chhattisgarh', 6)
(21, 'Haryana', 3)
(22, 'Madhya Pradesh', 6)
(23, 'Goa', 2)


In [ ]:
values = df
tablename = 'Food'

with engine.connect() as conn: 
    table = sqlalchemy.Table(tablename, metadata, autoload=True, autoload_with=engine)
    query = sqlalchemy.insert(table)
    for v in values.values:
        
        food_name = v[0]
        ingredients = v[1]
        diet_id = int(getId('Diet', v[2]))
        prep_time = int(v[3])
        cook_time = int(v[4])
        flavor_id = int(getId('Flavor_profile', v[5], col_id='flavor_id', col_name='flavor_name'))
        course_id = int(getId('Course', v[6]))
        state_id = int(getId('State', v[7]))
        region_id = int(getId('Region', v[8]))
        
        dic = {
            'food_name': food_name,
            'ingredients': ingredients,
            'diet_id': diet_id,
            'prep_time': prep_time,
            'cook_time': cook_time,
            'flavor_id': flavor_id,
            'course_id': course_id,
            'state_id': state_id,
            'region_id': region_id
        }
        
        conn.execute(query, dic)

In [ ]:
with engine.connect() as conn: 
    query = sqlalchemy.select([metadata.tables['Food']])
    result = conn.execute(query).fetchall()
    for r in result:
        print(r)

(1, 'Balu shahi', 'Maida flour, yogurt, oil, sugar', 45, 25, 1, 1, 1, 1, 1)
(2, 'Boondi', 'Gram flour, ghee, sugar', 80, 30, 2, 2, 1, 1, 1)
(3, 'Gajar ka halwa', 'Carrots, milk, sugar, ghee, cashews, raisins', 15, 60, 3, 3, 1, 1, 1)
(4, 'Ghevar', 'Flour, ghee, kewra, milk, clarified butter, sugar, almonds, pistachio, saffron, green cardamom', 15, 30, 2, 2, 1, 1, 1)
(5, 'Gulab jamun', 'Milk powder, plain flour, baking powder, ghee, milk, sugar, water, rose water', 15, 40, 1, 1, 1, 1, 1)
(6, 'Imarti', 'Sugar syrup, lentil flour', 10, 50, 1, 1, 1, 1, 1)
(7, 'Jalebi', 'Maida, corn flour, baking soda, vinegar, curd, water, turmeric, saffron, cardamom', 10, 50, 4, 3, 1, 1, 1)
(8, 'Kalakand', 'Milk, cottage cheese, sugar', 20, 30, 1, 1, 1, 1, 1)
(9, 'Lassi', 'Yogurt, milk, nuts, sugar', 5, 5, 3, 3, 1, 1, 1)
(10, 'Petha', 'Firm white pumpkin, sugar, kitchen lime, alum powder', 10, 30, 4, 3, 1, 1, 1)
(11, 'Phirni', 'Rice, sugar, nuts', 30, 20, 5, 1, 1, 1, 1)
(12, 'Rabri', 'Condensed milk, sugar